# EHDSLens Export & Reports

This notebook demonstrates how to generate reports and export bibliographies using EHDSLens.

In [ ]:
from ehdslens import EHDSAnalyzer
from ehdslens.export import ReportGenerator
from pathlib import Path

# Load data
analyzer = EHDSAnalyzer()
analyzer.load_default_data()

# Create report generator
reporter = ReportGenerator(analyzer.db)

print(f"Report generator ready with {len(analyzer.db)} studies")

## Generate Markdown Report

In [ ]:
# Generate comprehensive Markdown report
md_report = reporter.generate_markdown_report()

# Show first 2000 characters
print(md_report[:2000])
print("\n... [truncated] ...")

In [ ]:
# Save to file
# reporter.save_markdown_report(Path("ehds_report.md"))
print("To save: reporter.save_markdown_report(Path('ehds_report.md'))")

## Bibliography Generation

Export references in multiple citation formats:

In [ ]:
# BibTeX format
bibtex = reporter.generate_bibliography(format="bibtex")

print("=" * 50)
print("BIBTEX FORMAT (first 3 entries)")
print("=" * 50)

# Show first 3 entries
entries = bibtex.split("\n\n")[:3]
print("\n\n".join(entries))

In [ ]:
# RIS format (for EndNote, Zotero)
ris = reporter.generate_bibliography(format="ris")

print("=" * 50)
print("RIS FORMAT (first 2 entries)")
print("=" * 50)

# Show first 2 entries
entries = ris.split("ER  -\n\n")[:2]
print("ER  -\n\n".join(entries))

In [ ]:
# APA format
apa = reporter.generate_bibliography(format="apa")

print("=" * 50)
print("APA FORMAT (first 10 references)")
print("=" * 50)

for line in apa.split("\n")[:10]:
    print(line)

In [ ]:
# Vancouver format
vancouver = reporter.generate_bibliography(format="vancouver")

print("=" * 50)
print("VANCOUVER FORMAT (first 10 references)")
print("=" * 50)

for line in vancouver.split("\n")[:10]:
    print(line)

## JSON Export

In [ ]:
import json

# Export full dataset as JSON
json_data = reporter.export_json()

# Parse and show structure
data = json.loads(json_data)

print("JSON Export Structure:")
print(f"  - metadata: {list(data.get('metadata', {}).keys())}")
print(f"  - statistics: {list(data.get('statistics', {}).keys())}")
print(f"  - studies: {len(data.get('studies', []))} entries")

print("\nSample study entry:")
print(json.dumps(data['studies'][0], indent=2))

## Data Extraction Template

In [ ]:
# Generate data extraction template
template = reporter.generate_data_extraction_template()

print("=" * 60)
print("DATA EXTRACTION TEMPLATE")
print("=" * 60)
print(template[:1500])
print("\n... [truncated] ...")

## GRADE-CERQual Summary Report

In [ ]:
# Get GRADE-CERQual findings
findings = analyzer.get_grade_cerqual_summary()

print("=" * 70)
print("GRADE-CERQual SUMMARY OF FINDINGS")
print("=" * 70)

# Group by confidence level
by_confidence = {}
for f in findings:
    conf = f['confidence']
    if conf not in by_confidence:
        by_confidence[conf] = []
    by_confidence[conf].append(f)

confidence_order = ['high', 'moderate', 'low', 'very_low']
symbols = {'high': '🟢', 'moderate': '🟡', 'low': '🟠', 'very_low': '🔴'}

for conf in confidence_order:
    if conf in by_confidence:
        print(f"\n{symbols[conf]} {conf.upper()} CONFIDENCE")
        print("-" * 60)
        for f in by_confidence[conf]:
            print(f"• {f['finding']}")
            print(f"  (n={f['studies']} studies)")

## Custom Filtered Reports

In [ ]:
from ehdslens.data import ThematicAxis, QualityRating

# Filter high-quality governance studies
governance_studies = analyzer.filter_studies(
    axis=ThematicAxis.GOVERNANCE_RIGHTS_ETHICS,
    min_quality=QualityRating.HIGH
)

print("=" * 60)
print("HIGH-QUALITY GOVERNANCE STUDIES")
print("=" * 60)

for i, study in enumerate(governance_studies, 1):
    print(f"\n{i}. {study.authors} ({study.year})")
    print(f"   {study.title}")
    print(f"   {study.journal}")
    if study.doi:
        print(f"   DOI: {study.doi}")

In [ ]:
# Generate bibliography for filtered studies only
from ehdslens.data import StudyDatabase

# Create temporary database with filtered studies
filtered_db = StudyDatabase()
for study in governance_studies:
    filtered_db.add_study(study)

filtered_reporter = ReportGenerator(filtered_db)
filtered_bibtex = filtered_reporter.generate_bibliography(format="bibtex")

print("=" * 50)
print(f"BIBTEX for {len(governance_studies)} governance studies")
print("=" * 50)
print(filtered_bibtex[:1000])

## Export All Formats

In [ ]:
# Export all formats to files
output_dir = Path("exports")

print("Export commands:")
print(f"  reporter.save_markdown_report(output_dir / 'report.md')")
print(f"  reporter.save_bibliography(output_dir / 'references.bib', format='bibtex')")
print(f"  reporter.save_bibliography(output_dir / 'references.ris', format='ris')")
print(f"  reporter.save_json(output_dir / 'data.json')")

print("\nOr use CLI:")
print("  ehdslens export --format bibtex -o references.bib")
print("  ehdslens report -o report.md")